In [36]:
from sklearn.cluster import DBSCAN
import pandas as pd

In [105]:
df = pd.read_csv("namesmodifiedplayerdata.csv")

In [106]:
df.head()

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player-additional
0,0,1,"Achiuwa, Precious",C,22,TOR,73,1725,12.7,0.503,...,0.4,2.1,2.5,0.070,NaN,-2.0,-0.6,-2.6,-0.2,achiupr01
1,1,2,"Adams, Steven",C,28,MEM,76,1999,17.6,0.560,...,3.8,3.0,6.8,0.163,NaN,1.0,1.0,2.0,2.0,adamsst01
2,2,3,"Adebayo, Bam",C,24,MIA,56,1825,21.8,0.608,...,3.6,3.5,7.2,0.188,NaN,1.7,2.1,3.8,2.7,adebaba01
3,3,4,"Aldama, Santi",PF,21,MEM,32,360,10.2,0.452,...,-0.1,0.4,0.3,0.044,NaN,-4.2,-1.5,-5.7,-0.3,aldamsa01
4,4,5,"Aldridge, LaMarcus",C,36,BRK,47,1050,19.6,0.604,...,2.1,1.0,3.1,0.141,NaN,1.3,-0.6,0.7,0.7,aldrila01


In [89]:
df.columns

Index(['Unnamed: 0', 'Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48',
       'Unnamed: 24', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Player-additional'],
      dtype='object')

In [90]:
new_df = df[['MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]

In [91]:
X_train = new_df.fillna(new_df.mean())

In [92]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from kneed import KneeLocator

scaler = StandardScaler()
scaled_features = scaler.fit_transform(X_train)


kmeans_kwargs = {
 "init": "random",
 "n_init": 10,
   "max_iter": 300,
    "random_state": 42,
}

sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)


In [93]:
kl = KneeLocator(
    range(1, 11), sse, curve="convex", direction="decreasing"
)


In [94]:
kl.elbow

4

In [95]:
kmeans_kwargs = {
 "init": "random",
 "n_init": 10,
   "max_iter": 300,
    "random_state": 42,
}
kmeans = KMeans(n_clusters=4, **kmeans_kwargs)
kmeans.fit(scaled_features)

KMeans(init='random', n_clusters=4, n_init=10, random_state=42)

In [96]:
df.loc[:, 'Cluster'] = kmeans.labels_

In [97]:
df = df[['Player','Pos', 'MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Cluster']]

In [98]:
df = df.replace(['SG-PG', 'SG-SF', 'SF-SG', 'PF-SF',
       'C-PF', 'SG-PG-SF', 'PG-SG'], ['SG', 'SG', 'SF', 'PF', 'C', 'SG', 'PG'])
df['Pos'].value_counts()

SG    207
SF    157
PG    156
PF    151
C     141
Name: Pos, dtype: int64

In [99]:
df[df.Cluster==3]['PER'].mean()
#0 - guards and shooters, quick and agile
#1 -> versatile players, all-rounders, most efficient players and best players
#2 -> bigs, centers and power forwards
#3 -> weaker players, not heavy contributers

-6.6978723404255325

In [100]:
df[df.Cluster==3].describe()

,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Cluster
count,47.000000,47.000000,46.000000,46.000000,46.000000,47.000000,47.000000,47.000000,47.000000,47.000000,...,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.0
mean,21.510638,-6.697872,0.137087,0.511065,0.072022,5.231915,10.993617,8.106383,8.827660,1.719149,...,18.806383,-0.102128,0.006383,-0.063830,-0.277681,-13.242553,-4.059574,-17.308511,-0.055319,3.0
std,25.032911,9.217174,0.152070,0.343307,0.189105,10.095438,9.715931,7.696086,10.110965,4.305994,...,8.872562,0.084672,0.024709,0.079196,0.223622,5.907870,4.476492,7.736723,0.077484,0.0
min,2.000000,-45.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.400000,0.000000,-0.400000,-1.231000,-33.900000,-14.500000,-42.600000,-0.400000,3.0
25%,6.500000,-11.250000,0.000000,0.270750,0.000000,0.000000,0.000000,3.100000,0.000000,0.000000,...,13.850000,-0.100000,0.000000,-0.100000,-0.340000,-15.800000,-6.800000,-22.050000,-0.100000,3.0
50%,14.000000,-3.300000,0.105000,0.472000,0.000000,0.000000,10.900000,7.000000,4.900000,0.000000,...,18.400000,-0.100000,0.000000,-0.100000,-0.234000,-12.300000,-4.200000,-16.300000,0.000000,3.0
75%,25.000000,-0.150000,0.250000,0.787500,0.000000,5.650000,16.200000,11.400000,16.600000,1.200000,...,22.900000,-0.050000,0.000000,0.000000,-0.133500,-8.900000,-1.850000,-11.150000,0.000000,3.0
max,120.000000,3.700000,0.500000,1.000000,1.000000,48.900000,40.600000,44.700000,32.300000,25.000000,...,44.000000,0.000000,0.100000,0.000000,0.030000,-4.100000,11.500000,-7.400000,0.000000,3.0


In [101]:
df.describe()

,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Cluster
count,812.000000,812.000000,799.000000,797.000000,797.000000,812.000000,812.000000,812.000000,812.000000,812.000000,...,812.000000,812.000000,812.000000,812.000000,812.000000,812.000000,812.000000,812.000000,812.000000,812.000000
mean,825.200739,12.218719,0.527232,0.400491,0.248739,5.402709,14.759360,10.085345,12.868473,1.591872,...,18.014409,0.887685,0.821305,1.712808,0.065823,-1.998030,-0.403202,-2.399631,0.396675,0.820197
std,775.769997,7.749585,0.139421,0.236042,0.222867,5.665091,7.648939,5.636559,9.796030,1.731690,...,6.287915,1.534461,0.925846,2.291841,0.131728,4.440737,2.291294,5.746605,1.020179,0.972969
min,1.000000,-45.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-3.000000,-0.100000,-1.600000,-1.231000,-33.900000,-14.500000,-42.600000,-1.200000,0.000000
25%,121.000000,9.300000,0.495500,0.250000,0.126000,1.875000,10.500000,6.500000,6.600000,0.900000,...,14.300000,0.000000,0.100000,0.100000,0.031000,-3.525000,-1.300000,-4.325000,-0.100000,0.000000
50%,577.500000,12.500000,0.548000,0.403000,0.220000,3.500000,13.400000,8.700000,10.300000,1.400000,...,17.500000,0.200000,0.500000,0.800000,0.079000,-1.500000,-0.300000,-1.700000,0.000000,0.000000
75%,1414.500000,16.000000,0.593000,0.563000,0.314000,7.600000,18.700000,13.200000,17.300000,1.900000,...,21.300000,1.300000,1.200000,2.600000,0.124250,0.200000,0.600000,0.300000,0.500000,2.000000
max,2854.000000,76.200000,1.000000,1.000000,2.000000,56.600000,100.000000,54.500000,100.000000,25.000000,...,54.600000,10.800000,4.600000,15.200000,1.204000,31.000000,13.400000,36.900000,9.800000,3.000000


In [102]:
df.to_csv("clusteredplayers.csv")